### Import packages

In [1]:
import os
import glob
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import soundfile as sf
import yaml

### Choose file to inspect¶

In [2]:
directory='/Users/saroltagabulya/git/Orca/call_segments/'

folders=os.listdir(directory)
file=[]

dropdown=widgets.Dropdown(
    options=folders,
    description='CSV files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='CSV files:', options=('.DS_Store', '2017-02-04--10-25-15--00-05-25--C', '.ipynb_checkpoi…

2017-02-04--10-25-15--00-05-25--C


### Read wav file names

In [3]:
folder_name = directory + file[0]
os.chdir(folder_name)
wavs=glob.glob('*.wav')
wavs

['2017-02-04--10-28-15--00-00-03--C.wav',
 '2017-02-04--10-28-45--00-00-03--C.wav',
 '2017-02-04--10-29-32--00-00-04--C.wav',
 '2017-02-04--10-30-18--00-00-04--C.wav',
 '2017-02-04--10-28-22--00-00-04--C.wav',
 '2017-02-04--10-28-08--00-00-04--C.wav',
 '2017-02-04--10-28-03--00-00-01--C.wav',
 '2017-02-04--10-29-58--00-00-03--C.wav',
 '2017-02-04--10-28-36--00-00-03--C.wav',
 '2017-02-04--10-29-52--00-00-03--C.wav']

### Extract samplerate from metadata file

In [4]:
w=wavs[0]
yml_file=w[:-3] + 'yml'


with open(yml_file) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    metadata = yaml.load(file, Loader=yaml.FullLoader)

sample_rate=metadata['sample_rate']

### Functions for converting wavs into spectrograms

In [7]:
def open_wav(file, sample_rate):
    y, sr = librosa.load(file, sr=sample_rate)
    return y, sr

def wav_to_spect(y, sr, file): #, call_wav
    
    yml_file=file[:-3] + 'yml'


    with open(yml_file) as f:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
        metadata = yaml.load(f, Loader=yaml.FullLoader)

    ref=metadata['reference_value']
    raw_unit=metadata['raw_signal_unit']
    start_utc=metadata['start_utc']
    duration_seconds=metadata['duration_seconds']
    
    nfft=1024
    hop_length=32
    win_length=400
    
    if len(y)>0:
        # Convert to spectogram 
        
        Y = librosa.stft(y, n_fft=nfft, hop_length=hop_length, win_length=win_length) 
        Ydb = librosa.amplitude_to_db(abs(Y), ref=1)
        
        librosa.display.specshow(Ydb, cmap='seismic', sr=sr, x_axis='time', y_axis='fft', hop_length=32)
        
        cb=plt.colorbar()
        cb.set_label('Power [dB re to {} {}^2]'.format(ref, raw_unit), rotation=270, labelpad=+15)

        plt.xlabel('Time [s]')
        plt.ylabel('Frequency [kHz]'.format(ref, raw_unit))
        ticks, labels = plt.yticks()
        plt.yticks(ticks, [str(t / 1000) for t in ticks])
        plt.title('onset: {} (UTC) \nduration: {} seconds'.format(start_utc, duration_seconds))
        

        # Save spectograms
        plt.savefig(file[:-3] + 'png', bbox_inches='tight', dpi=600, facecolor='White')
        plt.close()
    
        return nfft, hop_length, win_length
    
    else: 
        return 'Error with wav'

### Iterate through wav files and save out spects

In [8]:
for w in list(range(len(wavs))): 
    print(w)
    wav=wavs[w]
    y, sr =open_wav(wav, sample_rate)
    nfft, hop_length, win_length= wav_to_spect(y, sample_rate, wav)

0
1
2
3
4
5
6
7
8
9


### Create spect_meta file

In [28]:
spect_meta={}

spect_meta['nfft']=nfft
spect_meta['hop_length']=hop_length
spect_meta['win_length']=win_length
spect_meta['sample_rate']=sr

In [29]:
with open('spect_meta.yml', 'w') as outfile:
    yaml.dump(spect_meta, outfile)